# 5. PDF-linkityskonversio

## 5.1 PDF-linkittäjä

### 5.1.1 Linkitystaulu

PDF-liitteiden linkittäminen yhtenäisessä muodossa kaavojen paikkatietoihin on mahdollista, jos tiedetään, mihin kaavaindeksitunnukseen kukin liite kytkeytyy. VOOKA-pilottiprojektissa tämä tieto puuttui lähes poikkeuksetta kaavojen paikkatietoaineistoista. Tästä syystä kaavaliitteiden ja niitä vastaavien kaavaindeksien tunnistustyö toteutettiin manuaalisena työnä.

Työssä PDF-linkitystä varten koostettiin liitostaulu, jossa jokaisella rivillä on tieto kaavan indeksitunnuksesta (KTJ, kunnan aineisto tai molemmat) sekä kaavan dokumenttityypistä (esim. osallistamis- ja arviointisuunnitelma). Kaiken kaikkiaan liitostaulun skeema oli seuraavanlainen:

| Ominaisuustieto | Selite |
| :--- | :--- |
| Kunta | Kunnan virallinen kuntakoodi |
| Kunnan indeksitunnus | Kunnan paikkatietomuotoisen kaava-aineiston kaavaindeksitunnus |
| KTJ-indeksitunnus | KTJ-aineiston kaavaindeksitunnus ("kaavatunnus_1") |
| Original filename | Kunnalta saadun kaavaliitteen alkuperäinen tiedostonimi |
| New filename | Sarake uudelle tiedostonimelle|
| Match equivalency % | Automaattisen linkityksen vastaavuusprosentti |
| Kaavalaji | Kaavalajikoodisto ylätasolla |
| Dokumentin tyyppi | Dokumenttityyppikoodisto, numeerinen |
| Huomioita | Tietoja liitteestä sanallisessa muodossa |
| Tila | Koodisto, joka kertoo, onko kaavaliite validi |
| Voimassa oleva | Onko liite voimassa vai ei (boolean) |
| Multipage | Onko liitteessä useampi kuin yksi sivu (boolean) |

Kaavalaji-koodisto:
- ak
- rak
- yk

Dokumentin tyyppi -koodisto (sanallinen ja numeerinen):
- 1 = kaavakartta (sis. merkinnät ja määräykset)
- 2 = kaavakartta (ei sis. merkinnät ja määräykset)
- 3 = merkinnät ja määräykset (erillisenä)
- 4 = kaavaselostus
- 5 = osallistamis- ja arviointisuunnitelma
- 6 = muu

Tila-koodisto:
- ok
- ei ok

### 5.1.2 Kaavaliitteiden hakemistorakenne

Kunnilta saadut kaavaliitteet tallennettiin vakioituun hakemistorakenteeseen resurssienhallintaan. Hakemistorakenteen runko oli muotoa:

```bash
documents   
│
└───kuntakoodi
│   │   
│   │
│   └───ak
│   │   liite.pdf
│   │   liite2.pdf
│   │   ...
│   └───rak
│   │   liite.pdf
│   │   liite2.pdf
│   │   ...
│   └───yk
│       liite.pdf
│       liite2.pdf
│       ...
...
```

**PDF-linkityskonversion ETL-funktiot on rakennettu kyseisen hakemistorakenteen varaan!**

### 5.1.3 Kaava-asiakirjojen lajittelu hakemistorakenteen mukaan kansioihin

Tämä koodi lajittelee PDF-asiakirjat hakemistorakenteen mukais. Koodi olettaa, että asiakirjat on lajiteltu kunnan nimellä olevaan kansioon kaavalajeittain. Mikäli kansio ei noudata tätä rakennetta, tulos on virheellinen.

Tarkoituksena on välttää kansioiden alikansioiden avaamisesta syntyvää lisätyötä sekä automaattisesti poimia vain PDF-muotoiset asiakirjat.

In [9]:
valittu_kansio = r'<insert filepath here>'


%run -i lib/sort_files_by_kaavalaji_main.py

Kansiolajittelu onnistui


### 5.1.3 Kaavaindeksien kansiolajittelu

In [2]:
valittu_kansio = r'<insert filepath here>'


%run -i lib/sort_index_files_by_kaavalaji_main.py

Kansiolajittelu onnistui


### 5.1.4 Multipage

<span style="color:red">**Tämä vaihe ei ole pakollinen!**</span>

ETL-työkalun avulla on mahdollista kertoa liitostaululle, mikäli kaavaliitteessä on enemmäni kuin yksi sivu.

In [ ]:
# Käytettävien Python moduulien ja kehitettyjen funktioiden sisäänluku
import pandas as pd
from lib.pdf_parser import declareMultipage

In [ ]:
# Liitostaulu
data = pd.read_csv(r"<insert filepath here>.csv", delimiter=',',  encoding='utf-8-sig') # vaihda delimiter muotoon ";" tarvittaessa

# Laskenta
multipage = declareMultipage(data=data,
                             master_dir=r"<insert filepath here>", # documents, koko hakemistorakenne
                             kuntakoodi="<insert kuntakoodi here>",
                             kaavalaji="ak") # hakemistorakenteen kaavalaji-kansionimi ("ak", "rak" tai "yk")

### 5.1.5 Kaava-asiakirjojen tiedostonimien linkitys kaavaindeksiaineistoon

<span style="color:red">**Tämä vaihe ei ole pakollinen! Vaatii kelvollisen paikkatietoaineiston indeksitietoineen sekä sen, että asiakirjat on nimetty sisältäen kaavaindeksin.**</span>

Mikäli kaava-asiakirjojen nimet sisältävät kunnan kaavaindeksin ja kunnasta on toimitettu paikkatietoaineisto, joka sisältää kaavaindeksin, on mahdollista linkittää kaava-asiakirjat indeksi aineistoon automaattisesti.

HUOM!
Automatisaatio perustuu sumeaan logiikkaan tarkoittaen sitä, että automatisaation linkittämät indeksit eivät välttämättä vastaa toisiaan täydellisesti.

Automatisaatio tuottaa CSV-taulukkoon arvon, joka ilmoittaa kuinka hyvin tiedostonimi vastasi indeksiä. Jos arvo on alle 80, niin tilanne tulee arvioida tilannekohtaisesti. Mitä matalampi arvo on, sitä todennäköisemmin automatisaatio on antanut väärän tai valheellisen linkitystuloksen.

HUOM!

Koodi tulkitsee dokumentin tyypin tiedostonimestä. Koodi antaa oletuksena dokumentin tyypiksi kaavakartan määräyksineen (arvo 1). Mikäli nimessä mainitaan merkintä tai määräys koodi antaa dokumentin tyypiksi kaavamääräyksen (3).
NÄMÄ TULEE AINA TARKISTAA MANUAALISESTI!

(Usein kaava-asiakirjojen tiedostonimessä voi esiintyä määräykseen viittaava sana, mutta todellisuudessa asiakirja sisältää sekä kartan että määräykset: esim. Kaava_1_kartta_merkinnat.pdf)

Koodin suorittamiseksi tarvitset kansion, jossa asiakirja-aineisto sijaitsee. Lisäksi tarvitset paikkatietomuotoisen indeksiaineiston.

Koodin suorittamiseksi anna:

-sen kunnan kuntanumero, jota haluat käsitellä.
-mitä kaavalajia olet käsittelemässä.
-missä sarakkeessa indeksitunnus sijaitsee paikkatiedossa.

Lisäksi anna koodiin se kansio, johon haluat valmiin CSV-tiedoston.

In [4]:
#Anna tiedostopolku kansioon missä kaava-asiakirjat sijaitsevat

kaava_asiakirjat = r'<insert filepath here>'


#Anna tiedostopolku tiedostonimineen kunnan indeksiaineistoon

#Indeksiaineiston tulee olla .gpkg tai .shp

kunta_kaavaindeksi = r'<insert filepath here>'

#Anna kuntanumero

kunta_numero = '<insert kuntanumero here>'

#Anna kaavalajin nimi

kaavalaji = '<insert kaavalaji here>'

#Anna sarakkeen nimi, jossa indeksitieto sijaitsee
indeksi_sarake = '<insert column name here>'

#Laita tähän sen kansion polku, johon haluat saada kaavatiedot.
directory = r'<insert kaavalaji here>'

#Suorita linkitysfunktio

%run -i lib/matching_filename_to_kaavaindex_main.py


Found 9 matches.
CSV file 'C:/Users/EmiliaTimlin/Documents/Meneillaan_olevat_Projektit/PohjoissavoVooka/testikansio_21//kaavatiedot_297_ak.csv' created successfully.


C:\Users\EmiliaTimlin\miniconda3\envs\VOOKAPS_dev\lib\site-packages\geopandas\array.py:85: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  aout[:] = out


Tuotoksena syntyy CSV-taulukko, joka vastaa PDF-linkityskonversiotaulukkoa.

| Kuntanumero | Kunnan indeksitunnus | KTJ-indeksitunnus | Original filename | New filename | Match equivalency %  | Kaavalaji | Manuaalisesti tarkistettu | Dokumentin tyyppi |
| :--- | :--- | :--- | :--- | :--- | :--- | :--- | :--- | :--- |
| "Kuntanumero" | "Kunnan kaavaindeksi" | "Tyhjä" | "Tiedoston nimi" | "Tyhjä" | "Vertailuarvo" | "Kaavalaji" | "FALSE" | "Dokumentin tyyppi" |
| 140* | 1* | - | Kaava_1.pdf | - | 90 | ak* | FALSE | 1* |

*esimerkki

### 5.1.6 Kaava-asiakirjataulun luominen PDF-linkityskonversiotauluun

<span style="color:red">**Mikäli vaihetta 5.1.5 ei voitu suorittaa aineistolle, voidaan suorittaa asiakirjataulun luonti tauluun ilman tiedostonimien linkitystä paikkatietoaineistoon.**</span>

Koodilla voidaan tuottaa useamman kunnan kaava-asiakirjat lajiteltuna taulukkoon.

HUOM! Tämä vaatii sen, että kohdan 5.1.2 ohjetta hakemistorakenteeseen on noudatettu.

In [2]:
#Anna tiedostopolku kansioon missä kaava-asiakirjat sijaitsevat
path_to_folder = r'<insert filepath here>'


#Laita tähän sen kansion polku, johon haluat saada kaavatiedot.
file_path = r'<insert filepath here>/'

%run -i lib/write_csv_main.py

CSV file C:/Users/EmiliaTimlin/Documents/Meneillaan_olevat_Projektit/PohjoissavoVooka/testikansio_21/kaavatiedot_297 created successfully.


### 5.1.7 Kaavaliitetiedostojen uudelleennimeäminen

Liitetiedostojen uudelleennimeämisen periaatteena on yhdenmukaisten liitenimien muodostus. ETL-työkalu muodostaa vakioidut liitenimet liitostauluun new_name-ominaisuustietokenttään seuraavin periaattein:

1) Liitenimi huomioi [virallisen kuntakoodin](https://www2.tilastokeskus.fi/fi/luokitukset/kunta/)
2) Liitenimi huomioi [Rakennetun ympäristön tietojärjestelmän (Ryhti) koodistojen mukaisesti kaavalajin](https://koodistot.suomi.fi/codescheme;registryCode=rytj;schemeCode=RY_Kaavalaji)
3) Liitenimi huomioi [Ryhti:n koodistojen mukaisesti asiakirjan lajin](https://koodistot.suomi.fi/codescheme;registryCode=rytj;schemeCode=RY_AsiakirjanLaji_YKAK)
4) Liitenimi huomioi kaavatunnuksen
5) Kaksi tai useampi liite ei saa olla samanniminen

Esimerkki ETL-työkalun tuottamasta liitenimestä on "491-31-05-13030-2.pdf", jossa:
- 491 on Mikkelin virallinen kuntakoodi
- 31 on Ryhti:n kaavalaji-koodiston mukaisesti asemakaava
- 05 viittaa Ryhti:n asiakirjan laji -koodiston mukaisesti siihen, että liite sisältää sekä kaavakartan että määräykset
- 13030 on kunnan indeksitunnus
- 2 on liukuva järjestysnumero (491-31-05-13030-1.pdf on jo olemassa, eli voimassa olevalle kaavalle on löytynyt kaksi samannimistä liitettä)

ETL-työkalu mahdollistaa liitetiedoston uudelleennimeämisen joko KTJ-indeksitunnukseen tai kunnan omaan indeksitunnukseen perustuen

<span style="color:red">**HUOM! Kaavaliitetiedostojen uudelleennimeämiskäytäntö on ainoastaan ehdotus liitteiden nimeämiselle Ryhti:ssä! Vakioitu nimeämiskäytäntö voi muuttua Ryhti-toteutusprojektin aikana!**</span>

In [ ]:
# Käytettävien Python moduulien ja kehitettyjen funktioiden sisäänluku
import geopandas as gpd
from lib.pdf_parser import createNewAttachmentName, renamePdfAttachments

In [ ]:
# Master geopackage
kaava_data_yk = gpd.read_file(r"<insert filepath here>.gpkg", layer="yleiskaavat_ktj") # vaihda kunnan aineisto tarvittaessa tilalle
kaava_data_ak = gpd.read_file(r"<insert filepath here>.gpkg", layer="asemakaavat_ktj") # vaihda kunnan aineisto tarvittaessa tilalle

# Voit myös yhdistää kaavadatat uudelleennimeämistä helpottaksesi.
# HUOM! Dataframe-skeemojen olisi syytä olla samat, jotta concat toimii järkevästi
kaava_data = pd.concat([kaava_data_ak, kaava_data_yk], ignore_index=True)

# Funktio palauttaa liitostaulun, jossa on liitteiden uudet nimet "New_name"-sarakkeessa
# Funktion voi ajaa monta kertaa samalle datalle eri parametrein (uusia, jo muodostettuja liitenimiä ei kirjoiteta yli)
data_newname = createNewAttachmentName(kaava_data=kaava_data,
                                       kaavadata_tunnus_column='kaavatunnus_1', # vaihda tarvittaessa "kaavatunnus" tilalle, jos haluat käyttää kunnan kaavatunnusta
                                       table_data=data, # liitostaulu
                                       table_tunnus_column='KTJ-indeksitunnus') # vaihda tarvittaessa "Kunnan indeksitunnus" tilalle

# Valitaan kunta, jonka liitteitä aletaan uudelleennimeämään
# Ei pakollista, mutta nopeuttaa uudeelleennimeämisprosessia
kunta = data_newname.loc[data_newname['Kunta'] == <insert kuntakoodi here>]

# Uudelleennimeäminen
renamePdfAttachments(data=kunta, # tai data_newname suoraan
                     master_dir=r"<insert filepath here>", # documents, koko hakemistorakenne
                     kuntakoodi="<insert kuntakoodi here>", # kuntakoodi-kansio hakemistorakenteessa
                     kaavalaji="ak") # vaihda "rak", "yk" uusiin ajoihin

### 5.1.8 Liitetiedostojen linkitys paikkatietoaineistoon

In [ ]:
# Kehitettyjen funktioiden sisäänluku
from lib.pdf_parser import joinPDFsToKaavadata

In [ ]:
# Esimerkkinä KTJ:n yleiskaavat
joined = joinPDFsToKaavadata(kaavadata=kaava_data_yk,
                             link_table=data_newname,
                             kuntakoodi="<insert kuntakoodi here>",
                             kaavalaji="yk", # kaavalaji hakemistorakenteessa, "ak", "rak" tai "yk"
                             kaavadata_tunnus_column="kaavatunnus_1", # vaihda tarvittaessa "kaavatunnus" tilalle, jos haluat käyttää kunnan kaavatunnusta
                             table_tunnus_column='KTJ-indeksitunnus') # vaihda tarvittaessa "Kunnan indeksitunnus" tilalle

# Tallennus
joined.to_file(r"<insert filepath here>.gpkg", layer='yleiskaavat_ktj', driver="GPKG")

## 5.2 PDF/A-konversio

ETL-työkalu konvertoi kunnilta saadut liite-PDF-tiedostot automatisoidusti Ghostscript-muuntimella PDF/A-arkistoformaattiin. Muunnin perustuu soveltuvin osin GitHubista avoimesti saatavilla olevaan [Python-Ghostscript-konvertteriin](https://github.com/Rajasekaran85/Python-PDF-A-Conversion). VOOKA-projektin tarpeisiin muokattu muunnin löytyy omana versionaan projektin [GitHub-sivuilta](https://github.com/ubigu/vooka).

Jos saatu PDF-formaatti on toteutettu versiolla, jota ei saada PDF/A-muotoon, se tallennetaan sellaisenaan. Mikäli jotkin kunnan PDF-tiedostot eivät käänny kunnan aineistosta johtuvista teknisistä syistä PDF/A-muotoon, tallennetaan nämä normaalissa PDF-muodossa.

ETL-työkalun PDF/A-konvertteri kysyy käyttäjältä tiedostopolkuja input- ja output-kansioihin. Muuntaja kääntää kaikki input-kansiossa olevat PDF-tiedostot PDF/A-formaattiin ja tallentaa ne output-kansioon.

In [ ]:
from lib.pdf_a_conv import writePdfA
writePdfA()